# Chapter 5: Formatting Output and Speaking for Claude

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic
import httpx

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME
%store -r PROXY_HTTP

proxy_client = httpx.Client(
    proxy=PROXY_HTTP
)

client = anthropic.Anthropic(
    api_key=API_KEY,
    http_client=proxy_client
)

# New argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

**Claude can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also ask Claude to **use XML tags to make its output clearer and more easily understandable** to humans.

### Examples

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling Claude to put the poem in XML tags**.

In [2]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Fluffy, twitching nose,
Hopping through the verdant grass,
Rabbit's gentle grace.
</haiku>


Why is this something we'd want to do? Well, having the output in **XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags**.

An extension of this technique is to **put the first XML tag in the `assistant` turn. When you put text in the `assistant` turn, you're basically telling Claude that Claude has already said something, and that it should continue from that point onward. This technique is called "speaking for Claude" or "prefilling Claude's response."

Below, we've done this with the first `<haiku>` XML tag. Notice how Claude continues directly from where we left off.

In [3]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly by my side,
Cat, my loyal friend.
</haiku>


Claude also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill Claude's response with the opening bracket, `{`}.

In [6]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------

  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly by my side,",
  "third_line": "Captivating cat."
}


Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [7]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Dearest Zack,

I do humbly beseech thee for a swift update on the prompt which thou wast charged to compose. Pray, let me know the progress of this endeavor, for I await thy response with bated breath.

Sincerely,
Thy humble servant
</olde english_email>


#### Bonus lesson

If you are calling Claude through the API, you can pass the closing XML tag to the `stop_sequences` parameter to get Claude to stop sampling once it emits your desired tag. This can save money and time-to-last-token by eliminating Claude's concluding remarks after it's already given you the answer you care about.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Change the `PREFILL` variable to **compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [10]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = "<answer>\nThat should be Steph Curry."

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN
<answer>
That should be Steph Curry.

------------------------------------- Claude's response -------------------------------------


Steph Curry is widely considered one of the greatest basketball players of all time. He has revolutionized the game with his incredible shooting ability and has led the Golden State Warriors to multiple NBA championships. Curry's combination of skill, athleticism, and basketball IQ make him a truly dominant and influential player who has left an indelible mark on the sport.
</answer>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [11]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write two haikus about {ANIMAL}. Put each in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write two haikus about cats. Put each in <haiku> tags.

ASSISTANT TURN
<haiku>

------------------------------------- Claude's response -------------------------------------

Feline grace and poise,
Purring softly in the sun,
Cats rule the domain.
</haiku>

<haiku>
Whiskers twitching, eyes
Watchful, stalking unseen prey,
Cats, masters of stealth.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### Exercise 5.3 - Two Haikus, Two Animals
Modify the `PROMPT` below so that **Claude produces two haikus about two different animals**. Use `{ANIMAL1}` as a stand-in for the first substitution, and `{ANIMAL2}` as a stand-in for the second substitution.

In [12]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write two haikus about {ANIMAL1} and {ANIMAL2} respectively. Put each in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write two haikus about Cat and Dog respectively. Put each in <haiku> tags.

------------------------------------- Claude's response -------------------------------------
Here are two haikus about Cat and Dog, each enclosed in <haiku> tags:

<haiku>
Feline grace and poise,
Purring softly by the fire,
Enigmatic cat.
</haiku>

<haiku>
Loyal companion,
Wagging tail, boundless energy,
Faithful, loving dog.
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))